In [1]:
import requests
import json
import pandas as pd
import time
import random
import xmltodict


In [9]:
def QueryEdit(url,keyword,Affiliation=None,daterange=None,start = 0,sort = 'citedby-count', count = 25):
    url +='?query={}'.format(keyword)
    if Affiliation!=None:
        url+="+AFFIL({})".format(Affiliation)
    url += "&count={}".format(count)
    url += "&sort={}".format(sort)
    url += "&start={}".format(start)
    return url

In [14]:
conf = open('config.json','r') 
config = json.load(conf)
api_key = config['API_key']
conf.close()
keyw = open('keywords.txt','r')
keywords = keyw.readlines()
keyw.close()
for index in range(len(keywords)):
    keywords[index] = keywords[index].replace('\n','')
print(keywords)
Affiliation = 'New Zealand'
start = 0
count = 25
step = 25
url = "https://api.elsevier.com/content/search/scopus"
headers=  {'X-ELS-APIKey':api_key}
print(url)


['waste reduction', 'maritime electrification']
https://api.elsevier.com/content/search/scopus


In [4]:
def TidyJsontoDf(web_json,paper_info_short,column):
    try :
        for paper in web_json['search-results']['entry']:
            try:
                title = paper['dc:title']
                autor = paper['dc:creator']
                doi = paper['prism:doi']
                citedby_count = paper['citedby-count']
                affilname = paper['affiliation'][0]['affilname']
                affiliation_city = paper['affiliation'][0]['affiliation-city']
                affiliation_country = paper['affiliation'][0]['affiliation-country']
            except:
                continue
            paper_data = [[title,autor,doi,citedby_count,affilname,affiliation_city,affiliation_country]]
            paper_info = pd.DataFrame(data=paper_data, columns=column)
            paper_info_short = pd.concat([paper_info_short,paper_info],ignore_index=True)
    except:
        pass
    return paper_info_short

In [18]:
def GetAndTidy(keyword,Affiliation = None,url=url, headers = headers,limit = 5000):
    url = QueryEdit(url,keyword,Affiliation = None,count = count)
    print(url)
    web = requests.get(url,headers=headers)
    result = web.text
    web_json = json.loads(result)
    print(web_json['search-results']['opensearch:totalResults'])
    epos = int(web_json['search-results']['opensearch:totalResults']) // step +1
    column = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
    paper_info_short = pd.DataFrame(columns=column)
    paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
    for i in range(1,epos+1):
        start = i*25
        if start > limit: break
        url = "https://api.elsevier.com/content/search/scopus"
        url = QueryEdit(url,keyword,start = start,count = count)
        print(url)
        web = requests.get(url,headers=headers)
        if(web.ok):
            result = web.text
            web_json = json.loads(result)
            paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
        # time.sleep(random.random())
    return paper_info_short



### Run one keyword

In [12]:
keyword = 'geothermal'
url = "https://api.elsevier.com/content/search/scopus"
paper_info = GetAndTidy(keyword,url = url)
paper_info.info()
filepath = 'data/'
paper_info.to_csv(filepath+keyword+'.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4738 entries, 0 to 4737
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                4738 non-null   object
 1   autor                4738 non-null   object
 2   doi                  4738 non-null   object
 3   citedby_count        4738 non-null   object
 4   affilname            4738 non-null   object
 5   affiliation-city     4712 non-null   object
 6   affiliation-country  4738 non-null   object
dtypes: object(7)
memory usage: 259.2+ KB


### Run all keywords

In [19]:
filepath = 'data/'
for keyword in keywords:
    url = "https://api.elsevier.com/content/search/scopus"
    paper_info = GetAndTidy(keyword,url = url)
    paper_info.to_csv(filepath+keyword+'.csv',index=False)

https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=0
453952
https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=25
https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=50
https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=75
https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=100
https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=125
https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=150
https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=175
https://api.elsevier.com/content/search/scopus?query=waste reduction&count=25&sort=citedby-count&start=200
https://api.elsevier.com/content/se

### NZ Data

In [17]:
paper_info.head()

,title,autor,doi,citedby_count,affilname,affiliation-city,affiliation-country
0,Dye-sensitized solar cells,Hagfeldt A.,10.1021/cr900356p,7630,Dalian University of Technology,Dalian,China
1,A review on polymer nanofibers by electrospinn...,Huang Z.M.,10.1016/S0266-3538(03)00178-7,6762,Tongji University,Shanghai,China
2,Chemistry and properties of nanocrystals of di...,Burda C.,10.1021/cr030063a,6692,Ctr. Chem. Dynamics/Nanomat. Res.,Cleveland,United States
3,The Li-ion rechargeable battery: A perspective,Goodenough J.,10.1021/ja3091438,6224,The University of Texas at Austin,Austin,United States
4,Properties of Polymers,Van Krevelen D.W.,10.1016/B978-0-08-054819-7.X0001-5,5846,Akzo Nobel N.V.,Amsterdam,Netherlands


### TEST

In [11]:
url = "https://api.elsevier.com/content/search/scopus"
keyword = 'additive manufacturing'
url = QueryEdit(url,keyword,Affiliation = None,count = 25)
print(url)
web = requests.get(url,headers=headers)
result = web.text
web_json = json.loads(result)
print(web_json)

https://api.elsevier.com/content/search/scopus?query=additive manufacturing&count=25&sort=citedby-count&start=0
{'search-results': {'opensearch:totalResults': '226972', 'opensearch:startIndex': '0', 'opensearch:itemsPerPage': '25', 'opensearch:Query': {'@role': 'request', '@searchTerms': 'additive manufacturing', '@startPage': '0'}, 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=additive+manufacturing&sort=citedby-count', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'first', '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=additive+manufacturing&sort=citedby-count', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'next', '@href': 'https://api.elsevier.com/content/search/scopus?start=25&count=25&query=additive+manufacturing&sort=citedby-count', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'last', '@href': 'https://api.elsevier.com/content/search/scopus?sta